In [131]:
import pandas as pd
import numpy as np

In [132]:
pl0 = pd.read_csv("Kraftwerksliste_CSV.csv", error_bad_lines=False, engine="python", encoding="latin1", sep=";", skiprows=9)
bm = pd.read_csv("block_plant_mapper.csv", error_bad_lines=False, engine="python")

In [133]:
pl0.rename(columns={ pl0.columns[10]: "Energieträger_old", pl0.columns[13]: "Energieträger"}, inplace=True)

In [134]:
#bm

In [135]:
#bm
pl1 = pl0.loc[np.logical_or(pl0.Energieträger == "Kernenergie", np.logical_or(pl0.Energieträger == "Erdgas", np.logical_or(pl0.Energieträger == "Steinkohle", pl0.Energieträger == "Braunkohle"))), :]

In [136]:
# pl1

In [137]:
pl2 = pl1.rename(columns={ pl1.columns[0]: "BlockID", pl1.columns[3]: "PLZ", pl1.columns[4]: "Ort", pl1.columns[8]: "Inbetriebnahme", pl1.columns[9]: "Status", pl1.columns[15]: "KWK", pl1.columns[16]: "Nettoleistung"})
#pl3 = pl2.rename(columns={ pl1.columns[1]: "BlockID" })

In [138]:
pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace('.','')
pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace(',','.')
#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace('\n','')
#pl2['Nettoleistung'] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace(';','.')

In [139]:
#pl2[] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
pl2['Nettoleistung'] = pl2['Nettoleistung'].apply(pd.to_numeric)

In [140]:
# pl2

In [141]:
cols = [10, 11,12,14,17,18,19]
pl3 = pl2.drop(pl2.columns[cols],axis=1)

In [142]:
# pl3

In [143]:
# Status: endgültig stillgelegt, vorläufig stillgelegt, in Betrieb, Sicherheitsbereitschaft, Sonderfall, 
# Stillegung: [Jahreszahl]
# StA: ja/ nein

In [144]:
newdf = pd.DataFrame()

In [145]:
def generate_decom_tuple(state):
    if state == "In Betrieb":
        return("in Betrieb", "0")
    elif "Endgültig Stillgelegt" in state:
        a = "stillgelegt"
        b = str(int(state[22:26]))
        return (a, b)
    elif "Vorläufig Stillgelegt" in state:
        a = "vorläufig stillgelegt"
        return (a, "0")
    else:
        return(state, "0")

In [146]:
def fix_company(company):
    cmp = company
    company_prefix, _ = company.split(" " , 1)
    # company_dict = {}
    company_dict = {"RWE": "RWE AG", "Vattenfall": "Vattenfall GmbH", "Uniper": "Uniper SE", "EnBW": "EnBW AG", "Steag": "Steag GmbH", "Nordzucker": "Nordzucker AG"}
    try:
        cmp = company_dict[company_prefix]
    except KeyError:
        return company
    return cmp

In [147]:
for index, row in pl3.iterrows():
    #if row['Status'] == "Endgültig Stillgelegt 2016 (mit StA)":
    # print(row)
    if row['KWK'] == "nein":
        row['KWK'] = "Nein" # workaround for BoA plants
    status, stilllegung = generate_decom_tuple(row['Status'])
    cmp = fix_company(row['Unternehmen'])
    row['Unternehmen'] = cmp
    row['Status'] = status
    row['Stilllegung'] = stilllegung
    newdf = newdf.append(row)
    

In [148]:
# newdf['Stilllegung'] = newdf['Stilllegung'].astype(int)
#newdf['Stilllegung'] = newdf['Stilllegung'].apply(lambda x: int(x) if x == x else None)

In [149]:
newdf['Stilllegung'] = newdf['Stilllegung'].replace("0", np.nan)

In [150]:
pl3 = newdf

In [151]:
atest = newdf.groupby(["Unternehmen"]).count()
atest.sort_values("Inbetriebnahme", inplace=True, ascending=False)

In [152]:
#print(atest)

In [153]:
atest = pl3.groupby(["Status"]).count()
# atest

In [154]:
# pl3.rename(columns={ pl3.columns[10]: "Energieträger"}, inplace=True)

In [155]:
pl4 = bm.merge(pl3, how="outer", on="BlockID")

In [156]:
pl5 = pl4[pd.notnull(pl4['BlockID'])]
#pl5 = pl4
pl6 = pl5[pd.notnull(pl5['Nettoleistung'])]

In [157]:
# pl6

In [158]:
blocks = pl6.copy()
stammdaten = pl6.copy()

In [159]:
blocks.sort_values("Inbetriebnahme", inplace=True, ascending=False)
# blocks

In [160]:
plants = blocks.loc[:]
plants = plants.dropna(subset=["KraftwerkID"])

In [161]:
plants_d = plants.loc[plants.duplicated("KraftwerkID", keep=False)]
# plants_d

In [162]:
plants_c = plants_d.groupby(["KraftwerkID"]).sum()
plants_count = plants_d.groupby(["KraftwerkID"]).count()

In [163]:
plants_f = plants_c.reset_index()

In [164]:
plants_count = plants_count.loc[:, ["BlockID"]]

In [165]:
plants_count = plants_count.reset_index()
# plants_count 

In [166]:
plants_f = plants_f.merge(plants_count, on="KraftwerkID", how="inner")
# plants_f

In [167]:
plants_a = plants.drop_duplicates(subset="KraftwerkID")

In [168]:
plants_b = plants_d.loc[:]

In [169]:
plants_f = plants_f.rename(columns={ "Nettoleistung": "Gesamtleistung", "BlockID": "Blockzahl"})

In [170]:
plants_f = plants_f.merge(plants_a, on="KraftwerkID", how="left")
# plants_f

In [171]:
blocks.drop_duplicates(subset="BlockID", inplace=True)

In [172]:
#cols = [4,5,6,8]
#blocks.drop(blocks.columns[cols],axis=1, inplace=True)
#test_blocks = blocks.drop(blocks.columns[cols],axis=1)

In [173]:
blocks.drop(['Ort', 'PLZ', 'Straße und Hausnummer (Standort Kraftwerk)', 'Blockname'], axis=1, inplace=True)

In [174]:
plants_f.sort_values("Inbetriebnahme", ascending=False, inplace=True)

In [175]:
drop_list = ['Blockname', 'KWK', 'Nettoleistung', 'Stilllegung', 'Straße und Hausnummer (Standort Kraftwerk)', 'PLZ', 'Ort', 'Nettoleistung']
plants_g = plants_f.drop(drop_list, axis=1)

In [176]:
plants_h = plants_g.rename(columns={"KraftwerkID": "plantid", "Gesamtleistung": "totalpower", "Blockzahl": "blockcount", "BlockID": "blockid", "Bundesland": "federalstate", "Energieträger": "energysource", "Inbetriebnahme": "latestexpanded", "Kraftwerksname": "plantname", "Status": "state", "Unternehmen": "company"})

In [177]:
plants_h.loc[plants_h.plantid == '06-10-0020157', 'latestexpanded'] = 2012

In [178]:
plants_h['latestexpanded'] = plants_h['latestexpanded'].astype(int)

In [179]:
plants_i = plants_h[["plantid", "plantname", "federalstate", "energysource", "latestexpanded", "totalpower", "state", "blockcount", "blockid", "company"]]

In [180]:
plants_h.shape

(92, 10)

In [181]:
plants_i.shape

(92, 10)

In [182]:
plants_h.dtypes

plantid            object
totalpower        float64
blockcount          int64
blockid            object
federalstate       object
energysource       object
latestexpanded      int64
plantname          object
state              object
company            object
dtype: object

In [183]:
# plants_i

In [184]:
# test_blocks

In [185]:
#blocks.loc[blocks.Energieträger == 'Erdgas',]

In [186]:
plants_final = plants_i.loc[:]

In [187]:
# plants_final

In [188]:
# stammdaten

In [189]:
# cols = [0,2,3,9,10,11,12]
drop_list = ['KraftwerkID', 'Blockname', 'Energieträger', 'Inbetriebnahme', 'KWK', 'Kraftwerksname', 'Nettoleistung', 'Status', 'Stilllegung', 'Unternehmen']
stammdaten.drop(drop_list, axis=1, inplace=True)

In [190]:
stammdaten.to_csv("stammdaten.csv", index=False)

In [191]:
plants_final.to_csv("plants.csv", index=False)

In [192]:
blocks.to_csv("blocks_3.csv", index=False)

In [193]:
stammdaten.to_csv("stammdaten_nh.csv", index=False, header=False)
blocks.to_csv("blocks_nh.csv", index=False, header=False)
plants_final.to_csv("plants_nh.csv", index=False, header=False)